## Import taxi_clean

In [27]:
import pandas as pd
import matplotlib as plt
import numpy as np
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

#import data 
data = pd.read_csv("taxi_clean_small.csv")

#display data
print(data.shape)
      
data.head()

(805, 19)


,trip_distance,fare_amount,winter,spring,summer,fall,PULongitude,PULatitude,DOLongitude,DOLatitude,pickup_datetime,dropoff_datetime,ride_duration,Early morning,Morning,Afternoon,Night,Holiday Proximity,label
0,0.80,9.50,0,1,0,0,-73.972145,40.756816,-73.977432,40.758226,2019-04-10 13:15:42,2019-04-10 13:30:15,0 days 00:14:33.000000000,0,0,1,0,0,C
1,0.50,4.50,0,0,0,1,-73.983310,40.737348,-73.985929,40.748808,2019-10-29 19:54:16,2019-10-29 19:58:23,0 days 00:04:07.000000000,0,0,0,0,0,A
2,1.21,9.50,1,0,0,0,-74.008386,40.735248,-73.994628,40.732486,2019-02-02 19:51:34,2019-02-02 20:05:11,0 days 00:13:37.000000000,0,0,0,0,0,C
3,8.77,29.89,0,0,0,1,-73.949270,40.667090,-73.988786,40.753514,2019-09-18 07:28:00,2019-09-18 08:14:00,0 days 00:46:00.000000000,0,1,0,0,0,J
4,1.95,11.50,1,0,0,0,-73.989857,40.762237,-73.990478,40.740313,2019-12-07 16:32:11,2019-12-07 16:48:18,0 days 00:16:07.000000000,0,0,1,0,0,D


In [28]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['pickup_datetime'] = pd.to_numeric(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])
data['dropoff_datetime'] = pd.to_numeric(data['dropoff_datetime'])

data['ride_duration'] = data['dropoff_datetime'] - data['pickup_datetime']

data.fillna(0, inplace= True)
print (data.dtypes)
data.head()

trip_distance        float64
fare_amount          float64
winter                 int64
spring                 int64
summer                 int64
fall                   int64
PULongitude          float64
PULatitude           float64
DOLongitude          float64
DOLatitude           float64
pickup_datetime        int64
dropoff_datetime       int64
ride_duration          int64
Early morning          int64
Morning                int64
Afternoon              int64
Night                  int64
Holiday Proximity      int64
label                 object
dtype: object


,trip_distance,fare_amount,winter,spring,summer,fall,PULongitude,PULatitude,DOLongitude,DOLatitude,pickup_datetime,dropoff_datetime,ride_duration,Early morning,Morning,Afternoon,Night,Holiday Proximity,label
0,0.80,9.50,0,1,0,0,-73.972145,40.756816,-73.977432,40.758226,1554902142000000000,1554903015000000000,873000000000,0,0,1,0,0,C
1,0.50,4.50,0,0,0,1,-73.983310,40.737348,-73.985929,40.748808,1572378856000000000,1572379103000000000,247000000000,0,0,0,0,0,A
2,1.21,9.50,1,0,0,0,-74.008386,40.735248,-73.994628,40.732486,1549137094000000000,1549137911000000000,817000000000,0,0,0,0,0,C
3,8.77,29.89,0,0,0,1,-73.949270,40.667090,-73.988786,40.753514,1568791680000000000,1568794440000000000,2760000000000,0,1,0,0,0,J
4,1.95,11.50,1,0,0,0,-73.989857,40.762237,-73.990478,40.740313,1575736331000000000,1575737298000000000,967000000000,0,0,1,0,0,D


## Random Forest

In [33]:
# Split dataset into training and test sets
y = data['label']
X = data.drop(columns=['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)


# Fit Random Forest classifier on training set
random_forest = RandomForestClassifier(n_estimators=200, max_depth=5, max_features= 4, random_state=42)

# Add Grid
param_grid = {'n_estimators': [200],'max_features': ['auto', 'sqrt', 'log2'], 'max_depth' : [4,5,6,7,8],'criterion' :['gini', 'entropy']}
new_tune = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv= 5)
new_tune.fit(X_train,y_train)
c_score3 = cross_val_score(new_tune, X_train, y_train, cv=2)
print('Accuracy of classifier: ', c_score3.mean())

print('\n Best value for each of the tested parameters:',new_tune.best_params_)
print('\n and accuracy of the model with these best values is',new_tune.best_score_)

C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will

Accuracy of classifier:  0.8451498300895891

 Best value for each of the tested parameters: {'criterion': 'entropy', 'max_depth': 8, 'max_features': 'auto'}

 and accuracy of the model with these best values is 0.8757763975155279


C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


"# Pass training data into clasifier\nclf = new_tune.fit(X_train, y_train)\n\n#Classify Test data\ny_pred = clf.predict(X_test)\n\n# Print Confusion Matrix\nprint('\n',confusion_matrix(y_test,y_pred))\n\n# Priny confusion matrix\nprint('\n',classification_report(y_test,y_pred))\n\n# Print accuracy score\nprint('\n',accuracy_score(y_test, y_pred))"

In [45]:
rfc1=RandomForestClassifier(random_state=42, criterion = 'entropy', max_depth = 8, max_features = 'auto')
rfc1.fit(X_train, y_train)
pred=rfc1.predict(X_test)
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

Accuracy for Random Forest on CV data:  0.8757763975155279


C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [46]:
# Print confusion matrix 
print('\n Confusion Matrix \n',confusion_matrix(y_test,pred))

# Print classification report
print('\n Classification report \n',classification_report(y_test,pred))


 Confusion Matrix 
 [[ 41   5   0   0   0   0   0   0   0   0]
 [  1 167   1   0   0   0   0   0   0   0]
 [  0   3 129   2   0   0   0   0   0   0]
 [  0   0   4  84   5   0   0   0   0   0]
 [  0   0   0  14  31   4   0   0   0   0]
 [  0   0   1   0   8  36   0   0   0   0]
 [  0   0   0   0   0   3  26   0   2   1]
 [  0   0   0   0   0   0   8   2   1   1]
 [  0   0   0   0   0   1   2   2   0   3]
 [  0   0   0   0   0   1   3   2   2  48]]

 Classification report 
               precision    recall  f1-score   support

           A       0.98      0.89      0.93        46
           B       0.95      0.99      0.97       169
           C       0.96      0.96      0.96       134
           D       0.84      0.90      0.87        93
           E       0.70      0.63      0.67        49
           F       0.80      0.80      0.80        45
           G       0.67      0.81      0.73        32
           H       0.33      0.17      0.22        12
           I       0.00      0.00  

## Decision Trees

In [47]:
# Fit a decision tree classifier on the training set
decision_tree = DecisionTreeClassifier(random_state=0)

# Run Cross Validation loop and print accuracy
c_score = cross_val_score(decision_tree, X_train, y_train, cv=10)
print('Accuracy of classifier: ', c_score.mean())

# Add Grid
new_grid = {'max_depth':[5,10,15,20], 'min_samples_leaf':[5,10,15,20], 'max_features':[5,10,15]}
new_tune1 = GridSearchCV(decision_tree,param_grid=new_grid,scoring='accuracy',cv=10)

c_score2 = cross_val_score(new_tune1, X_train, y_train, cv=10)
print('Accuracy of classifier: ', c_score2.mean())

# Pass training data to classifier
new_tune1.fit(X_train, y_train)

print('\n Best value for each of the tested parameters:',new_tune1.best_params_)
print('\n and accuracy of the model with these best values is',new_tune1.best_score_)

# Run test data and predict classes
y_pred1 = new_tune1.predict(X_test)

# Print confusion matrix 
print('\n Confusion Matrix \n',confusion_matrix(y_test,y_pred1))

# Print classification report
print('\n Classification report \n',classification_report(y_test,y_pred1))

# Print accuracy score
print('\n Accuracy is:',accuracy_score(y_test, y_pred1))

C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Accuracy of classifier:  0.9769502808404411


C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class 

Accuracy of classifier:  0.952366034826662

 Best value for each of the tested parameters: {'max_depth': 10, 'max_features': 15, 'min_samples_leaf': 5}

 and accuracy of the model with these best values is 0.9440993788819876

 Confusion Matrix 
 [[ 46   0   0   0   0   0   0   0   0   0]
 [  0 169   0   0   0   0   0   0   0   0]
 [  0   0 134   0   0   0   0   0   0   0]
 [  0   0   0  93   0   0   0   0   0   0]
 [  0   0   0   0  49   0   0   0   0   0]
 [  0   0   0   0   0  45   0   0   0   0]
 [  0   0   0   0   0   0  32   0   0   0]
 [  0   0   0   0   0   0   2   0  10   0]
 [  0   0   0   0   0   0   0   0   8   0]
 [  0   0   0   0   0   0   0   0   0  56]]

 Classification report 
               precision    recall  f1-score   support

           A       1.00      1.00      1.00        46
           B       1.00      1.00      1.00       169
           C       1.00      1.00      1.00       134
           D       1.00      1.00      1.00        93
           E       1.00   

C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\maniv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
